In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from collections import Counter
from subprocess import check_output
#看看文件是否存在，并且是不是用utf8
print(check_output(["ls", "input"]).decode("utf8"))

In [ ]:
#孩子喜欢的礼物
gift_pref = pd.read_csv('input/child_wishlist_v2.csv',header=None).drop(0, 1).values
#礼物喜欢的孩子
child_pref = pd.read_csv('input/gift_goodkids_v2.csv',header=None).drop(0, 1).values

In [ ]:
#有1000000个children
n_children = 1000000 
#1000种gift
n_gift_type = 1000
#每种gift1000个
n_gift_quantity = 1000 
#每个孩子有喜欢的100个礼物。(孩子开心)
n_gift_pref = 100 # number of gifts a child ranks
#每个礼物优先考虑1000个好孩子（圣诞老人开心）
n_child_pref = 1000 # number of children a gift ranks
#0-5000为三胞胎，triplets=50001 0.5%
triplets = math.ceil(0.005 * n_children / 3.) * 3    
#5001-45000双胞胎，即45000-5001+1=40000
twins = math.ceil(0.04 * n_children / 2.) * 2    
#根据目标函数里面定义的
ratio_gift_happiness = 2
ratio_child_happiness = 2

In [ ]:
#三胞胎孩子
def Triplet_Hij(Children,prefer1, prefer2, prefer3):
    res=[[],[],[]]
    for p in list(set(list(prefer1) + list(prefer2) + list(prefer3))):        
        score = 0
        #对于三胞胎中的第一个人，偏爱有各种礼物的得分
        if p in list(prefer1):
            score += 2*(100 - list(prefer1).index(p))
        else:
            score -= 1
        #对于三胞胎中的第二个人
        if p in list(prefer2):
            score += 2*(100 - list(prefer2).index(p))
        else:
            score -= 1
        #对于三胞胎中的第三个人
        if p in list(prefer3):
            score += 2*(100 - list(prefer3).index(p))
        else:
            score -= 1
        res[0].append(p)
        res[1].append(2*score)
        res[2].append(-6)
    Children.append(res)
    Children.append(res)
    Children.append(res)

In [ ]:
def Twin_Hij(Children,prefer1, prefer2):
    res=[[],[],[]]
    for p in list(set(list(prefer1) + list(prefer2))):
        score = 0
        if p in list(prefer1):
            score += 2*(100 - list(prefer1).index(p))
        else:
            score -= 1
        if p in list(prefer2):
            score += 2*(100 - list(prefer2).index(p))
        else:
            score -= 1
        res[0].append(p)
        res[1].append(3*score)
        res[2].append(-6)
    Children.append(res)
    Children.append(res) 

In [ ]:
#普通孩子子
def Child_Hij(Children,prefer):
    res=[[],[],[]]
    for i in range(prefer.shape[0]):
        score = 12*(prefer.shape[0] - i)
        res[0].append(prefer[i])
        res[1].append(score)
        res[2].append(-6)
    Children.append(res)

In [ ]:
#处理孩子偏爱的礼物
Children1=[]
for i in range(0, 5001, 3):
    Triplet_Hij(Children1,gift_pref[i], gift_pref[i+1], gift_pref[i+2])
for i in range(5001, 45001, 2):
    Twin_Hij(Children1,gift_pref[i], gift_pref[i+1])
for i in range(45001, 1000000):
    Child_Hij(Children1,gift_pref[i])

In [ ]:
def index_of(in_list,val):
    try:
        return in_list.index(val)
    except ValueError:
        return -1 

In [ ]:
#处理礼物偏爱的孩子
for j in range(1000):
    #每类礼物更偏爱的孩子
    cf = child_pref[j]
    done_list = []
    for i in range(cf.shape[0]):
        #同一礼物如果是三胞胎则以三胞胎中最先出现的为主。
        if cf[i] <= 5000 and cf[i] not in done_list:
            if cf[i] % 3 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                cid3 = cf[i] + 2
                done_list.append(cid2)
                done_list.append(cid3)
            elif cf[i] % 3 == 1:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                cid3 = cf[i] + 1
                done_list.append(cid1)
                done_list.append(cid3)
            else:
                cid1 = cf[i] - 2
                cid2 = cf[i] - 1
                cid3 = cf[i]
                done_list.append(cid1)
                done_list.append(cid2)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            if cid3 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid3))
            else:
                score_ += -1
            #判断孩子(cid1)是否偏爱礼物(j)
            index_triplet_pref=index_of(Children[cid1][0],j)
            #存在修改
            if index_triplet_pref>-1:
                #修改Hji
                Children[cid1][2][index_triplet_pref]=2*score_
                Children[cid2][2][index_triplet_pref]=2*score_
                Children[cid3][2][index_triplet_pref]=2*score_
            else:
                #要添加
                Children[cid1][1].append(-6)
                Children[cid2][1].append(-6)
                Children[cid3][1].append(-6)
                Children[cid1][2].append(2*score_)
                Children[cid2][2].append(2*score_)
                Children[cid3][2].append(2*score_)         
            #如果是双胞胎
            #以双胞胎最先出现的为主
        elif cf[i] <= 45000 and cf[i] not in done_list:
            if cf[i] % 2 == 1:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                done_list.append(cid2)
            else:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                done_list.append(cid1)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            #判断孩子(cid1)是否偏爱礼物(j)
            index_twin_pref=index_of(Children[cid1][0],j)
            #存在修改
            if index_twin_pref>-1:
                Children[cid1][2][index_twin_pref]=3*score_
                Children[cid2][2][index_twin_pref]=3*score_
            else:
                Children[cid1][1].append(-6)
                Children[cid2][1].append(-6)
                Children[cid1][2].append(3*score_)
                Children[cid2][2].append(3*score_)
            #如果是普通孩子
        elif cf[i] > 45000:
            #判断孩子(cid1)是否偏爱礼物(j)
            cid=cf[i]
            score_=2*(cf.shape[0] - i)
            index_child_pref=index_of(Children[cid][0],j)
            if index_child_pref>-1:
                Children[cid][2][index_child_pref]=6*score_
            else:
                Children[cid][1].append(-6)
                Children[cid][2].append(6*score_)

### 验证更改后是否正确(注：其中Children是我改的，Children1是原来的）
```python
import random
def validat_mychange(cid):
    #判断giftid的数量是否相等
    print(len(Children1[cid].prefer_dict.keys()))
    print(len(Children[cid][0]))
    #判断里面的值是否相等，随机取的一个
    random_giftid=random.choice(Children[cid][0])
    print(random_giftid)
    print(Children1[cid].prefer_dict[random_giftid])
    print(Children[cid][1][Children[cid][0].index(random_giftid)])
    print(Children[cid][2][Children[cid][0].index(random_giftid)])
```

### Summarize
至此有关Children的处理已经完毕，需要改进的是里面Score的计算。以后你们可以用本页中的Children了，我已经验证了准确性。速度加快了不少。
与原来的Children对比详见：Compare_Children.ipynb

### 下面的还未改完，晚点再改，我这都改完了就只剩下里面的参数调节了，和最大限度运用目标函数，将Score分摊到Cost上，这是创新点也是难点。

### 最小Cost最大流

In [ ]:
from ortools.graph import pywrapgraph

In [ ]:
W_CHILD = 10000
W_GIFTS = 0

In [ ]:
min_cost_flow_1 = pywrapgraph.SimpleMinCostFlow()

start_nodes = []
end_nodes = []
capacities = []
unit_costs = []

In [ ]:
#构建StartNodes
# triplets
for i in range(0, 5001, 3):
    for g in Children[i][0]:
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(3)
        giftid_index=index_of(Children[i][0],g)
        unit_costs.append(-W_CHILD*(Children[i][1][giftid_index] + 6)-W_GIFTS*(Children[i][2][giftid_index] + 6))
        
# triplets
for i in range(5001, 45001, 2):
    for g in Children[i][0]:
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(2)
        giftid_index=index_of(Children[i][0],g)
        unit_costs.append(-W_CHILD*(Children[i][1][giftid_index] + 6)-W_GIFTS*(Children[i][2][giftid_index] + 6))
        
# other children
for i in range(45001, 1000000):
    
    for g in Children[i][0]:
        start_nodes.append(1000000+g)
        end_nodes.append(i)
        capacities.append(1)
        giftid_index=index_of(Children[i][0],g)
        unit_costs.append(-W_CHILD*(Children[i][1][giftid_index] + 6)-W_GIFTS*(Children[i][2][giftid_index] + 6))


In [ ]:
#开始画图了
# add Arc（画边，用上面的start_nodes-->end_nodes,即gift-->childern)
# gift -> children
for i in range(len(start_nodes)):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )


In [ ]:
# 1001000结点为孩子节点的集合结点，添加从每个收到礼物的孩子结点到此节点的一条路径，capacity=礼物数，cost为0,Good!
# children -> 1001000 : collection(孩子到终止节点)

for i in range(0, 5001, 3):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(3), int(0)
    )
for i in range(5001, 45001, 2):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(2), int(0)
    )
for i in range(45001, 1000000):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(i), int(1001000), int(1), int(0)
    )

In [ ]:
#gift结点到终止节点capacity=1000 and cost=0
# gift -> 1001001 : dust_gift
# 1001001结点为礼物结点的汇总节点，添加从每个礼物结点到汇总节点的容量为1000花费为0的通路
for i in range(1000):
    min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(1000000+i), int(1001001), int(1000), int(0)
    )
    

In [ ]:
# 1001001 -> 1001000 : dust_path
# 添加从开始结点的汇总节点到结束结点的汇总节点的容量为1000000花费为0的通路
min_cost_flow_1.AddArcWithCapacityAndUnitCost(
        int(1001001), int(1001000), int(1000000), int(0)
    )

In [ ]:
# add gift Supply
for i in range(1000):
    min_cost_flow_1.SetNodeSupply(int(1000000+i), int(1000))

In [ ]:
# add children Supply
for i in range(0, 5001, 3):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(5001, 45001, 2):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))
for i in range(45001, 1000000):
    min_cost_flow_1.SetNodeSupply(int(i), int(0))

In [ ]:
#孩子的终点处supply为-1000000
min_cost_flow_1.SetNodeSupply(int(1001000), int(-1000000)) 
#gift的终点处supply为0
min_cost_flow_1.SetNodeSupply(int(1001001), int(0)) 

In [ ]:
assignment = [-1]*1000000
twins_differ = []
triplets_differ = []

for i in range(min_cost_flow_1.NumArcs()):
    if min_cost_flow_1.Flow(i) != 0 and min_cost_flow_1.Head(i) < 1000000:
        c = min_cost_flow_1.Head(i)
        g = min_cost_flow_1.Tail(i)
        f = min_cost_flow_1.Flow(i)

        if c >= 45001:
            assignment[c] = g - 1000000

        elif c >= 5001:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    twins_differ.append([c, c+1])
                else:
                    assignment[c+1] = g - 1000000
            elif f == 2:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
        else:
            if f == 1:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                elif assignment[c+1] == -1:
                    assignment[c+1] = g - 1000000
                else:
                    assignment[c+2] = g - 1000000
            elif f == 2:
                if assignment[c] == -1:
                    assignment[c] = g - 1000000
                    assignment[c+1] = g - 1000000
                    triplets_differ.append([c, c+1, c+2])
                else:
                    assignment[c+1] = g - 1000000
                    assignment[c+2] = g - 1000000
            elif f == 3:
                assignment[c] = g - 1000000
                assignment[c+1] = g - 1000000
                assignment[c+2] = g - 1000000
   
#assignment[i]好像表示的是礼物id,还没细看
CHILD_HAPPINESS = sum([Children[i].happiness(assignment[i])[0] for i in range(1000000)])*10
SANTA_HAPPINESS = sum([Children[i].happiness(assignment[i])[1] for i in range(1000000)])
OBJ = CHILD_HAPPINESS**3 + SANTA_HAPPINESS**3
print(W_CHILD, W_GIFTS, CHILD_HAPPINESS, SANTA_HAPPINESS, OBJ)
print(OBJ / (12000000000**3))
